In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "request": "I want to know about Your Name movie.",
        "answer": """
About the movie:

Genre: Animation, Fantasy, Romance
Director: Makoto Shinkai 
main casts: Taki, Mitsuha
budget: 750 million yen
box office gross: 380 million dollars
synopsis: Taki and Mitsuha, two high school students, mysteriously switch bodies and uncover a life-changing secret as their connection deepens.
""",
    },
    {
        "request": "I want to know about Howl's Moving Castle movie.",
        "answer": """
About the movie:

Genre: Animation, Fantasy, Romance
Director: Hayao Miyazaki
Main Casts: Sophie, Howl, Calcifer
Budget: 2.4 billion yen
Box Office Gross: 235 million dollars
Synopsis: Sophie, cursed by a witch to become an old woman, seeks refuge in Howl’s magical moving castle and becomes entangled in his battle against dark forces.
""",
    },
    {
        "request": "I want to know about Suzume no Tojimari movie.",
        "answer": """
About the movie:

Genre: Animation, Fantasy, Drama
Director: Makoto Shinkai
Main Casts: Suzume, Sota
Budget: 2 billion yen
Box Office Gross: 250 million dollars
Synopsis: Suzume, a young girl, embarks on a journey with a mysterious young man to close doors that are causing disasters across Japan.
""",
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {request}\n AI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: I want to know about {moviename} movie.",
    input_variables=["moviename"],
)

chain = prompt | chat
chain.invoke({"moviename": "Frozen"})

AI:
About the movie:

Genre: Animation, Adventure, Comedy
Directors: Chris Buck, Jennifer Lee
Main Casts: Elsa, Anna, Olaf, Kristoff
Budget: $150 million
Box Office Gross: $1.28 billion
Synopsis: Frozen follows the story of two sisters, Elsa and Anna, as Elsa struggles to control her magical ice powers and inadvertently plunges their kingdom into eternal winter. Anna sets out on a journey to find her sister and save their kingdom with the help of a rugged iceman, a comedic snowman, and a loyal reindeer.

AIMessageChunk(content='AI:\nAbout the movie:\n\nGenre: Animation, Adventure, Comedy\nDirectors: Chris Buck, Jennifer Lee\nMain Casts: Elsa, Anna, Olaf, Kristoff\nBudget: $150 million\nBox Office Gross: $1.28 billion\nSynopsis: Frozen follows the story of two sisters, Elsa and Anna, as Elsa struggles to control her magical ice powers and inadvertently plunges their kingdom into eternal winter. Anna sets out on a journey to find her sister and save their kingdom with the help of a rugged iceman, a comedic snowman, and a loyal reindeer.')